In [1]:
cd /SNS/users/lj7/dv/sns-chops/resolution/ARCS/

/SNS/users/lj7/dv/sns-chops/resolution/ARCS


In [2]:
#Program to load Vanadium or empty Can powder files and perform a constant-Q cut along the middle Q to look at energy resolution and intensity.
import matplotlib.pyplot as plt
from mantid import plots
from mantid.simpleapi import Load, ConvertToMD, BinMD, ConvertUnits, Rebin
from matplotlib.colors import LogNorm
import numpy as np
import mantid.simpleapi as msa
from mantid import mtd

In [3]:
%matplotlib notebook

In [4]:
RunNumbers=range(113843,113970+1)
print(RunNumbers)
IPTS=21387
datadir="/SNS/ARCS/IPTS-"+str(IPTS)+"/shared/autoreduce/"

[113843, 113844, 113845, 113846, 113847, 113848, 113849, 113850, 113851, 113852, 113853, 113854, 113855, 113856, 113857, 113858, 113859, 113860, 113861, 113862, 113863, 113864, 113865, 113866, 113867, 113868, 113869, 113870, 113871, 113872, 113873, 113874, 113875, 113876, 113877, 113878, 113879, 113880, 113881, 113882, 113883, 113884, 113885, 113886, 113887, 113888, 113889, 113890, 113891, 113892, 113893, 113894, 113895, 113896, 113897, 113898, 113899, 113900, 113901, 113902, 113903, 113904, 113905, 113906, 113907, 113908, 113909, 113910, 113911, 113912, 113913, 113914, 113915, 113916, 113917, 113918, 113919, 113920, 113921, 113922, 113923, 113924, 113925, 113926, 113927, 113928, 113929, 113930, 113931, 113932, 113933, 113934, 113935, 113936, 113937, 113938, 113939, 113940, 113941, 113942, 113943, 113944, 113945, 113946, 113947, 113948, 113949, 113950, 113951, 113952, 113953, 113954, 113955, 113956, 113957, 113958, 113959, 113960, 113961, 113962, 113963, 113964, 113965, 113966, 113967,

# gather data

In [5]:
PlotTag=0

data_table = {}

for RunNumber in RunNumbers:
    print (RunNumber)
    try:
        w=msa.CreateSingleValuedWorkspace()
        #LoadNexusLogs(w,"/SNS/ARCS/IPTS-"+str(IPTS)+"/data/ARCS_"+str(RunNumber)+'_event.nxs')
        msa.LoadNexusLogs(w,"/SNS/ARCS/IPTS-"+str(IPTS)+"/nexus/ARCS_"+str(RunNumber)+'.nxs.h5')
        RunParams=w.getRun()
        Energy=RunParams["BL18:Chop:Skf0:EnergyUserReq"].getStatistics().mean
        #print Energy
        Chopperpos=RunParams["chtrans"].getStatistics().mean
        #print Chopperpos
        Chopper1=RunParams["Speed1"].getStatistics().mean
        Chopper2=RunParams["Speed2"].getStatistics().mean
        Chopper3=RunParams["Speed3"].getStatistics().mean
        if Chopperpos>400: 
            Chopper=2 
        elif Chopperpos<1: 
            Chopper=1 
        else: 
            Chopper=0

        
        # generate a nice 2D multi-dimensional workspace
        data = msa.LoadNXSPE(datadir+'ARCS_'+str(RunNumber)+'_autoreduced.nxspe')
        values=msa.ConvertToMDMinMaxLocal('data',QDimensions='|Q|', dEAnalysisMode='Direct')
        minQ,minE=values.MinValues
        maxQ,maxE=values.MaxValues
        
        md = msa.ConvertToMD(InputWorkspace=data, QDimensions='|Q|', dEAnalysisMode='Direct')
        sqw = msa.BinMD(InputWorkspace=md,
                    AlignedDim0='|Q|,'+str(minQ)+','+str(maxQ)+',100',
                    AlignedDim1='DeltaE,'+ str(minE) +',' +str(maxE*0.8) +',100')

        #2D plot
        if PlotTag==1:
            fig, ax = plt.subplots(subplot_kw={'projection':'mantid'})
            c = ax.pcolormesh(sqw, vmin=0., vmax=0.5e-3)
            cbar=fig.colorbar(c)
            cbar.set_label('Intensity (arb. units)') #add text to colorbar
            ax.set_title('Run '+str(RunNumber)+',Ei='+str(Energy)+'meVChoppers=['+str(Chopper1)+','+str(Chopper2)+','+str(Chopper3)+']')
            fig.show()


        # generate a 1D multi-dimensional workspace


        sqw_line= msa.BinMD(
            InputWorkspace=md,
            #AlignedDim0='|Q|,' +str((minQ+maxQ)/3-0.01*maxQ) +','+ str((minQ+maxQ)/3+0.01*maxQ) +',1',
            AlignedDim0='|Q|,%s,%s,1' % ((minQ*2+maxQ)/3 , (minQ+2*maxQ)/3),
            AlignedDim1='DeltaE,'+ str(minE) +',' +str(maxE*0.8) +',100')

        sqw_line_Hist=msa.ConvertMDHistoToMatrixWorkspace('sqw_line', Normalization='NumEventsNormalization')

        # plots 1D multi-dimensional workspace
        if PlotTag==1:
            fig, ax = plt.subplots(subplot_kw={'projection':'mantid'})
            c = ax.errorbar(sqw_line_Hist,label='Data')
            ax.legend()
            ax.set_title('Run '+str(RunNumber)+',Ei='+str(Energy)+'meV,|Q|=[' +str((minQ+maxQ)/3-0.01*maxQ) +','+ str((minQ+maxQ)/3+0.01*maxQ) +']' )
            fig.show()
        xbb, y, Err = sqw_line_Hist.extractX(), sqw_line_Hist.extractY(), sqw_line_Hist.extractE()
        xbb.shape = y.shape = Err.shape = -1,
        x = (xbb[1:]+xbb[:-1])/2
        IE = x,y,Err

        RunParams2=data.getRun()
        Ei=RunParams2["Ei"].value
        Q=(minQ+maxQ)/3
        # array=[0,RunNumber, Energy, Ei, Chopper, round(Chopper1), round(Chopper2), round(Chopper3), Height, dHeight, Center, dCenter, Sigma, dSigma, Q]
        print("Run=",RunNumber,", Energy=",Energy,"meV, Chopper=",Chopper,"ChopperPosition=",Chopperpos)
        #print array
        data_table[RunNumber] = [
            Energy, Ei, 
            Chopper, round(Chopper1), round(Chopper2), round(Chopper3),
            IE,
            Q]
        
    except:
        raise
        print(RunNumber,": Errors, either file not in directory or some syntax, or fitting / plot errors")


113843
('Run=', 113843, ', Energy=', 2000.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113844
('Run=', 113844, ', Energy=', 1500.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113845
('Run=', 113845, ', Energy=', 1000.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113846
('Run=', 113846, ', Energy=', 700.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113847
('Run=', 113847, ', Energy=', 500.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113848
('Run=', 113848, ', Energy=', 2000.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113849
('Run=', 113849, ', Energy=', 1500.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113850
('Run=', 113850, ', Energy=', 1000.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113851
('Run=', 113851, ', Energy=', 700.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113852
('Run=', 113852, ', Energy=', 500.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113853
('Run=', 113853, ', Energy=

('Run=', 113928, ', Energy=', 150.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113929
('Run=', 113929, ', Energy=', 125.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113930
('Run=', 113930, ', Energy=', 100.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113931
('Run=', 113931, ', Energy=', 90.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113932
('Run=', 113932, ', Energy=', 150.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113933
('Run=', 113933, ', Energy=', 125.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113934
('Run=', 113934, ', Energy=', 100.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113935
('Run=', 113935, ', Energy=', 90.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113936
('Run=', 113936, ', Energy=', 150.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113937
('Run=', 113937, ', Energy=', 125.0, 'meV, Chopper=', 1, 'ChopperPosition=', 0.67002012)
113938
('Run=', 113938, ', Energy=', 100.0, 'meV,

# Directly get FWHM using interpolation

In [7]:
import scipy

In [8]:
%%time
plot = False
rows = []
for RunNumber, record in data_table.items():
    Energy, Ei, \
        Chopper, Chopper1, Chopper2, Chopper3,\
        IE,\
        Q = record
    x,y,Err = IE
    x_interp = np.arange(-Ei/2, Ei/2, Ei/1000)
    # y_interp = np.interp(x_interp, x,y)
    y_interp2 = scipy.interpolate.interp1d(x,y, kind='quadratic', bounds_error=False)(x_interp)

    # print Ei, Energy
    if plot:
        plt.figure()
        plt.plot(x, y, '+')
        plt.plot(x_interp, y_interp2, 'o')
    ymax = np.nanmax(y_interp2)
    hm = ymax/2
    middle = x_interp[y_interp2>hm]
    if middle.size:
        FWHM = middle[-1]-middle[0]
    else:
        FWHM = -1.
    Height = ymax
    Sigma = FWHM/2.355
    row=[
        0,RunNumber, 
        Energy, Ei, 
        Chopper, Chopper1, Chopper2, Chopper3,
        Height, FWHM, Sigma, Q
    ]
    rows.append(row)
    continue

CPU times: user 62.7 ms, sys: 2.92 ms, total: 65.6 ms
Wall time: 63.9 ms


/SNS/software/miniconda2/envs/mcvine-unstable/lib/python2.7/site-packages/ipykernel/__main__.py:17: RuntimeWarning: invalid value encountered in greater
/SNS/software/miniconda2/envs/mcvine-unstable/lib/python2.7/site-packages/ipykernel/__main__.py:15: RuntimeWarning: All-NaN slice encountered


In [9]:
np.savetxt(
    "./V_Cali_Int_Res_FC1_HighRes_2018_v2.dat",
    rows,
    header='RunNumber Energy Ei Chopper Chopper1 Chopper2 Chopper3 Height FWHM Sigma Q'
)